In [1]:
import sys
if not 'informer' in sys.path:
    sys.path += ['informer']

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
import torch

In [3]:
args = dotdict()

args.model = 'informer' # model of experiment, options: [informer, informerstack, informerlight(TBD)]

args.data = 'WaterLevel' # data
args.root_path = './dataset/' # root path of data file
args.data_path = 'WaterLevel.csv' # data file
args.features = 'MS' # forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate
args.target = 'sw' # target feature in S or MS task
args.freq = 'h' # freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h
args.checkpoints = './informer_checkpoints' # location of model checkpoints

args.seq_len = 96 # input sequence length of Informer encoder
args.label_len = 48 # start token length of Informer decoder
args.pred_len = 24 # prediction sequence length
# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

args.enc_in = 7 # encoder input size
args.dec_in = 7 # decoder input size
args.c_out = 1 # output size
args.factor = 5 # probsparse attn factor
args.d_model = 512 # dimension of model
args.n_heads = 8 # num of heads
args.e_layers = 2 # num of encoder layers
args.d_layers = 1 # num of decoder layers
args.d_ff = 2048 # dimension of fcn in model
args.dropout = 0.05 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = False # whether to output attention in ecoder
args.mix = True
args.padding = 0
args.freq = 'h'

args.batch_size = 32 
args.learning_rate = 0.0001
args.loss = 'mse'
args.lradj = 'type1'
args.use_amp = False # whether to use automatic mixed precision training

args.num_workers = 0
args.itr = 1
args.train_epochs = 6
args.patience = 3
args.des = 'exp'

args.use_gpu = True if torch.cuda.is_available() else False
args.gpu = 0

args.use_multi_gpu = False
args.devices = '0,1,2,3'

In [4]:
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ','')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

In [5]:
# Set augments by using data name
data_parser = {
    'ETTh1':{'data':'ETTh1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTh2':{'data':'ETTh2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTm1':{'data':'ETTm1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTm2':{'data':'ETTm2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
}
if args.data in data_parser.keys():
    data_info = data_parser[args.data]
    args.data_path = data_info['data']
    args.target = data_info['T']
    args.enc_in, args.dec_in, args.c_out = data_info[args.features]

In [6]:
args.detail_freq = args.freq
args.freq = args.freq[-1:]

In [7]:
print('Args in experiment:')
print(args)

Args in experiment:
{'model': 'informer', 'data': 'WaterLevel', 'root_path': './dataset/', 'data_path': 'WaterLevel.csv', 'features': 'MS', 'target': 'sw', 'freq': 'h', 'checkpoints': './informer_checkpoints', 'seq_len': 96, 'label_len': 48, 'pred_len': 24, 'enc_in': 7, 'dec_in': 7, 'c_out': 1, 'factor': 5, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'dropout': 0.05, 'attn': 'prob', 'embed': 'timeF', 'activation': 'gelu', 'distil': True, 'output_attention': False, 'mix': True, 'padding': 0, 'batch_size': 32, 'learning_rate': 0.0001, 'loss': 'mse', 'lradj': 'type1', 'use_amp': False, 'num_workers': 0, 'itr': 1, 'train_epochs': 6, 'patience': 3, 'des': 'exp', 'use_gpu': False, 'gpu': 0, 'use_multi_gpu': False, 'devices': '0,1,2,3', 'detail_freq': 'h'}


In [8]:
Exp = Exp_Informer

In [9]:
args.train_epochs = 1

In [10]:
args

{'model': 'informer',
 'data': 'WaterLevel',
 'root_path': './dataset/',
 'data_path': 'WaterLevel.csv',
 'features': 'MS',
 'target': 'sw',
 'freq': 'h',
 'checkpoints': './informer_checkpoints',
 'seq_len': 96,
 'label_len': 48,
 'pred_len': 24,
 'enc_in': 7,
 'dec_in': 7,
 'c_out': 1,
 'factor': 5,
 'd_model': 512,
 'n_heads': 8,
 'e_layers': 2,
 'd_layers': 1,
 'd_ff': 2048,
 'dropout': 0.05,
 'attn': 'prob',
 'embed': 'timeF',
 'activation': 'gelu',
 'distil': True,
 'output_attention': False,
 'mix': True,
 'padding': 0,
 'batch_size': 32,
 'learning_rate': 0.0001,
 'loss': 'mse',
 'lradj': 'type1',
 'use_amp': False,
 'num_workers': 0,
 'itr': 1,
 'train_epochs': 1,
 'patience': 3,
 'des': 'exp',
 'use_gpu': False,
 'gpu': 0,
 'use_multi_gpu': False,
 'devices': '0,1,2,3',
 'detail_freq': 'h'}

In [11]:
for ii in range(args.itr):
    # setting record of experiments
    setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features, 
                args.seq_len, args.label_len, args.pred_len,
                args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, args.embed, args.distil, args.mix, args.des, ii)

    # set experiments
    exp = Exp(args)
    
    # train
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    exp.train(setting)
    
    # test
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    exp.test(setting)

    torch.cuda.empty_cache()

Use CPU
>>>>>>>start training : informer_WaterLevel_ftMS_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
hello world
WaterLevel.csv
yesyes
[[-1.8765661  -1.84561753  0.95691572 ...  0.16568732 -0.16816948
  -0.06583949]
 [-1.87651196 -1.84561753  0.90338417 ... -1.19006135 -0.16816948
  -0.06583949]
 [-1.87645782 -1.84561753  0.90338417 ...  0.16568732 -0.16816948
  -0.09942972]
 ...
 [ 1.82182246  0.66472748 -0.06018366 ... -1.19006135 -0.16816948
  -0.87200498]
 [ 1.8218766   0.66472748 -0.00665211 ... -1.19006135 -0.16816948
  -0.87200498]
 [ 1.82193074  1.08311832 -0.43490448 ...  0.16568732 -0.16816948
  -0.80482452]]
[[-1.8765661  -1.84561753  0.95691572 ...  0.16568732 -0.16816948
  -0.06583949]
 [-1.87651196 -1.84561753  0.90338417 ... -1.19006135 -0.16816948
  -0.06583949]
 [-1.87645782 -1.84561753  0.90338417 ...  0.16568732 -0.16816948
  -0.09942972]
 ...
 [ 1.82182246  0.66472748 -0.06018366 ... -1.19006135 -0.1681694

NameError: name 'exit' is not defined

In [12]:
data = pd.read_csv("base_info\WaterLevel.csv", encoding='utf-8')
plt.figure(figsize=(20,5))
plt.plot(data["sw"])
plt.show()
data.head()

NameError: name 'pd' is not defined